In [48]:
import pandas as pd 
import tensorflow 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
from datetime import datetime 
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle 

label_encoder_gender = LabelEncoder()
one_hot_encoder = OneHotEncoder(sparse_output = False)


In [49]:
salary = pd.read_csv('Churn_modelling.csv')
salary = salary.drop(columns = ['RowNumber', 'CustomerId', 'Surname'])
salary.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [50]:
cols = salary.select_dtypes(include = 'object').columns 

salary[cols] = salary[cols].astype('category')

In [51]:
idsx = salary.select_dtypes(include = 'category')
for i in idsx:
    if len(salary[i].cat.categories) <= 2:
        salary[i] = label_encoder_gender.fit_transform(salary[i])
    else:
        salary[i] = salary[i]

In [52]:
geo_df = one_hot_encoder.fit_transform(salary[['Geography']])

geo_df = pd.DataFrame(geo_df, columns = one_hot_encoder.get_feature_names_out(['Geography']))
salary = pd.concat([salary.drop(columns = 'Geography'), geo_df], axis = 1)

In [53]:
salary

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [54]:
y = salary['EstimatedSalary']
x = salary.drop(columns = ['EstimatedSalary'])

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)


In [55]:
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.mean_absolute_error

def create_model(neurons = 32, layers = 1):
    model = Sequential()
    model.add(Dense(neurons, activation = 'relu', input_shape = (x_train.shape[1],)))

    for _ in range(layers - 1):
        model.add(Dense(neurons, activation = 'relu'))

    model.add(Dense(1))
    model.compile(optimizer = opt, loss = loss, metrics = ['mae'])

    return model

In [56]:
model = KerasRegressor(layers = 1, neurons = 32, build_fn = create_model, verbose = 1)

In [57]:
param_grid = {
    'neurons': [16, 32, 64, 128],
    'layers': [1, 2, 3],
    'batch_size': [10, 20],
    'epochs': [50, 100]
}

In [58]:
grid = GridSearchCV(estimator = model, param_grid = param_grid, n_jobs = -1, cv = 3)
grid_result = grid.fit(x_train, y_train)

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

c:\Users\tariq\Desktop\Neural Network Project\venv\Lib\site-packages\scikeras\wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/100
800/800 [==============================] - 1s 1ms/step - loss: 99556.5078 - mae: 99556.5078
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 94193.8984 - mae: 94193.8984
Epoch 3/100
800/800 [==============================] - 1s 972us/step - loss: 85735.0312 - mae: 85735.0312
Epoch 4/100
800/800 [==============================] - 1s 960us/step - loss: 76654.3750 - mae: 76654.3828
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 68343.5078 - mae: 68343.5078
Epoch 6/100
800/800 [==============================] - 1s 2ms/step - loss: 61663.7695 - mae: 61663.7695
Epoch 7/100
800/800 [==============================] - 1s 2ms/step - loss: 56758.0312 - mae: 56758.0312
Epoch 8/100
800/800 [==============================] - 1s 2ms/step - loss: 53416.6211 - mae: 53416.6211
Epoch 9/100
800/800 [==============================] - 1s 969us/step - loss: 51464.9258 - mae: 51464.9258
Epoch 10/100
800/800 [==============================] - 1s

In [68]:
model = Sequential([
    Dense(16, activation = 'relu', input_shape = (x_train.shape[1],)),
    Dense(1)

])

In [69]:
opt = tensorflow.keras.optimizers.Adam(learning_rate = 0.01)
loss = tensorflow.keras.losses.mean_absolute_error

In [70]:
model.compile(optimizer = opt, loss = loss, metrics = ['mae'])

In [71]:
log_dir = 'regressions/fit/' + datetime.now().strftime('%m_%d_%Y_%S_%M_%H')
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)
early_stopping_callback = EarlyStopping(monitor = 'val_loss', patience = 10, restore_best_weights = True)

In [72]:
history = model.fit(
    x_train, y_train, validation_data = (x_test, y_test), epochs  = 100, batch_size = 10, 
    callbacks = [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
800/800 [==============================] - 1s 1ms/step - loss: 99561.1406 - mae: 99561.1406 - val_loss: 95440.3438 - val_mae: 95440.3438
Epoch 2/100
800/800 [==============================] - 1s 1ms/step - loss: 94120.9297 - mae: 94120.9297 - val_loss: 88036.5547 - val_mae: 88036.5547
Epoch 3/100
800/800 [==============================] - 1s 2ms/step - loss: 85472.3516 - mae: 85472.3516 - val_loss: 78765.9062 - val_mae: 78765.9062
Epoch 4/100
800/800 [==============================] - 1s 1ms/step - loss: 76033.6719 - mae: 76033.6719 - val_loss: 69705.4141 - val_mae: 69705.4141
Epoch 5/100
800/800 [==============================] - 1s 1ms/step - loss: 67377.6016 - mae: 67377.6016 - val_loss: 62181.1562 - val_mae: 62181.1562
Epoch 6/100
800/800 [==============================] - 1s 1ms/step - loss: 60441.9062 - mae: 60441.9062 - val_loss: 56737.6836 - val_mae: 56737.6836
Epoch 7/100
800/800 [==============================] - 1s 1ms/step - loss: 55504.2852 - mae: 55504.2852 - 

In [73]:
model.save('model2.h5')

c:\Users\tariq\Desktop\Neural Network Project\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [74]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [75]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 44596), started 1 day, 6:40:37 ago. (Use '!kill 44596' to kill it.)

In [67]:
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f'Test Mae: {test_mae}')

63/63 [==============================] - 0s 969us/step - loss: 51056.0234 - mae: 51056.0234
Test Mae: 51056.0234375
